# Least-Cost Path Routing using REMA DEM
This notebook performs routing based on slope-derived cost from REMA mosaics. It:
- Loads a shapefile AOI
- Queries the PGC STAC API for matching REMA tiles
- Computes slope and a cost surface
- Runs least-cost routing between two coordinate points
- Saves outputs as GeoTIFFs

## 1. Imports
Load all necessary geospatial, raster, and routing libraries.

In [1]:
import geopandas as gpd  # for vector data handling
from shapely.geometry import mapping  # convert geometries to GeoJSON-like dicts
from pystac_client import Client  # access STAC API
import stackstac  # turn STAC items into xarray objects
import rioxarray  # adds raster and CRS support to xarray
import numpy as np  # numerical processing
import xarray as xr  # labeled N-D arrays
from rasterio.transform import rowcol  # convert coordinates to pixel indices
from skimage.graph import route_through_array  # least-cost path routing

## 2. Load AOI
Load the Area of Interest shapefile and remove a problematic feature.

In [2]:
aoi = gpd.read_file("data/bat_boundary/bat_boundary.shp")  # load AOI shapefile
aoi_cleaned = aoi.drop(index=53).reset_index(drop=True)  # drop corrupted polygon

## 3. STAC Search and Stack
Query REMA mosaics intersecting the AOI from the PGC STAC API.

In [3]:
catalog = Client.open("https://stac.pgc.umn.edu/api/v1/")

search = catalog.search(
    collections=["rema-mosaics-v2.0-32m"],
    intersects=mapping(aoi.geometry[9]),  # area selected to minimize computational cost
    datetime="2009-01-01/2025-06-30"
)
items = search.item_collection()
print(f"Found {len(items)} REMA items")

Found 4 REMA items


In [5]:
# Convert the REMA items into a stacked xarray.DataArray (lazy loaded)
stack = stackstac.stack(
    items,
    epsg=3857,
    resolution=32
).rio.write_crs("EPSG:3857")  # ensure CRS is set

# Extract maximum elevation over time
dem = stack.sel(band="dem").max(dim="time")

(4, 6, 19471, 15879)
Full data loaded


## 4. Slope and Cost Surface
Derive slope from the DEM and generate a cost surface for routing.

In [8]:
# Get spatial resolution (in projected units, e.g., meters)
res_y, res_x = map(abs, dem.rio.resolution())

# Calculate gradients along x and y directions
dz_dx, dz_dy = np.gradient(dem.values, res_y, res_x)

# Compute slope magnitude and convert to degrees
slope_degrees = np.degrees(np.arctan(np.sqrt(dz_dx**2 + dz_dy**2)))

Slope calculated


In [9]:
# Wrap slope into xarray.DataArray for metadata + spatial reference
slope = xr.DataArray(
    slope_degrees,
    coords=dem.coords,
    dims=dem.dims,
    attrs=dem.attrs
).rio.write_crs(dem.rio.crs)

# Normalize slope and create cost surface: higher slope = higher cost
slope_normalized = (slope - slope.min()) / (slope.max() - slope.min())
cost_surface = 1 + (slope_normalized * 9)  # scale cost between 1–10
cost_surface.rio.write_crs(dem.rio.crs)

# Save outputs
slope.rio.to_raster("slope_degrees.tif")
cost_surface.rio.to_raster("cost_surface.tif")

## 5. Routing Setup
Define start/end points and translate them into array indices.

In [10]:
print("Beginning routing")

# Start and end coordinates in EPSG:3857 (meters)
start_coords = (-7508109, -10237837)
end_coords   = (-7467806, -10283841)

# Convert geographic coordinates to array row/col
transform = slope.rio.transform()
start_row, start_col = rowcol(transform, *start_coords)
end_row, end_col = rowcol(transform, *end_coords)

Beginning routing


In [11]:
# Replace NaNs and infinities in cost array to avoid routing errors
cost_array = np.nan_to_num(cost_surface.values, nan=9999, posinf=9999, neginf=9999)

## 6. Pathfinding
Use `route_through_array` to find the least-cost path.

In [12]:
indices, total_cost = route_through_array(
    cost_array,
    (start_row, start_col),
    (end_row, end_col),
    fully_connected=True
)

# Convert list of indices to binary mask (1 for path, 0 elsewhere)
path_mask = np.zeros_like(cost_array, dtype=np.uint8)
for r, c in indices:
    path_mask[r, c] = 1

path_da = xr.DataArray(
    path_mask,
    coords=slope.coords,
    dims=slope.dims,
    attrs=slope.attrs
)

## 7. Export Results
Save least-cost path to a GeoTIFF file.

In [13]:
path_da.rio.to_raster("least_cost_path.tif")